In [4]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import os
import matplotlib.pylab as plt
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import sklearn.metrics
 # Feature Importance
from sklearn import datasets
from sklearn.ensemble import ExtraTreesClassifier

### Load the dataset

In [5]:
columns = ["H1CO1","H1MO1","H1MO2","H1MO3","H1MO4","H1MO5","H1MO8","H1MO9","H1MO10","H1MO11","H1MO13","H1RE4","H1RE6","H1BC8","H1BC7","BIO_SEX"]

AH_data = pd.read_csv("../../data/addhealth_pds.csv", usecols=columns)

filter_answers = [6,8,9]

data_filter1 = AH_data.dropna()[~AH_data['H1CO1'].isin(filter_answers)]

data_clean = data_filter1[~data_filter1['H1MO4'].isin([6,7,8,9])]

data_clean.dtypes

BIO_SEX    int64
H1MO1      int64
H1MO2      int64
H1MO3      int64
H1MO4      int64
H1MO5      int64
H1MO8      int64
H1MO9      int64
H1MO10     int64
H1MO11     int64
H1MO13     int64
H1CO1      int64
H1BC7      int64
H1BC8      int64
H1RE4      int64
H1RE6      int64
dtype: object

## Modeling and Prediction

### Split into training and testing sets

In [6]:
predictors = data_clean[["H1MO1","H1MO2","H1MO3","H1MO4","H1MO5","H1MO8","H1MO9","H1MO10","H1MO11","H1MO13","H1RE4","H1RE6","H1BC8","H1BC7","BIO_SEX"]]

targets = data_clean.H1CO1

pred_train, pred_test, tar_train, tar_test  =   train_test_split(predictors, targets, test_size=.4)


### Build model on training data

In [7]:
from sklearn.ensemble import RandomForestClassifier

classifier=RandomForestClassifier(n_estimators=25)
classifier=classifier.fit(pred_train,tar_train)

predictions=classifier.predict(pred_test)

sklearn.metrics.confusion_matrix(tar_test,predictions)
sklearn.metrics.accuracy_score(tar_test, predictions)


# fit an Extra Trees model to the data
model = ExtraTreesClassifier()
model.fit(pred_train,tar_train)
# display the relative importance of each attribute
importances = model.feature_importances_
var_imp = pd.DataFrame(data = {'predictors':list(predictors.columns.values),'importances':importances})
print(var_imp.sort_values(by='importances', ascending=False))

[ 0.06085028  0.07110611  0.10857838  0.0804474   0.07028096  0.06640975
  0.06474151  0.07373985  0.06514039  0.05523976  0.05501062  0.06210014
  0.07301819  0.05986994  0.03346674]


In [22]:
#list(predictors.columns.values)
importances = model.feature_importances_
var_imp = pd.DataFrame(data = {'predictors':list(predictors.columns.values),'importances':importances})
print(var_imp.sort_values(by='importances', ascending=False))

    importances predictors
2      0.108578      H1MO3
3      0.080447      H1MO4
7      0.073740     H1MO10
12     0.073018      H1BC8
1      0.071106      H1MO2
4      0.070281      H1MO5
5      0.066410      H1MO8
8      0.065140     H1MO11
6      0.064742      H1MO9
11     0.062100      H1RE6
0      0.060850      H1MO1
13     0.059870      H1BC7
9      0.055240     H1MO13
10     0.055011      H1RE4
14     0.033467    BIO_SEX


In [24]:
trees=range(25)
accuracy=np.zeros(25)

for idx in range(len(trees)):
   classifier=RandomForestClassifier(n_estimators=idx + 1)
   classifier=classifier.fit(pred_train,tar_train)
   predictions=classifier.predict(pred_test)
   accuracy[idx]=sklearn.metrics.accuracy_score(tar_test, predictions)
   
plt.cla()
plt.plot(trees, accuracy)

